### Étude particulière du dataset "users-score-2023.csv" :

In [1]:
import pandas as pd
import gc
from surprise import Dataset, Reader, accuracy, SVD, SVDpp
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.slope_one import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from sklearn.neighbors import NearestNeighbors

In [2]:
# Charger le dataset
user_score = pd.read_csv("./users-score-2023.csv")

In [3]:
# Aperçu du dataset
print(user_score.head())
print(user_score.info())

   user_id Username  anime_id             Anime Title  rating
0        1    Xinil        21               One Piece       9
1        1    Xinil        48             .hack//Sign       7
2        1    Xinil       320                  A Kite       5
3        1    Xinil        49        Aa! Megami-sama!       8
4        1    Xinil       304  Aa! Megami-sama! Movie       8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int64 
 1   Username     object
 2   anime_id     int64 
 3   Anime Title  object
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 927.9+ MB
None


### Restreindre le dataset

In [4]:
# Fixer les seuils
min_user_ratings = 50  # Par exemple, les utilisateurs doivent avoir au moins noté 50 animes
min_anime_ratings = 10_000  # Par exemple, les animes doivent avoir au moins 10 000 notes

# Compter le nombre de notes par utilisateur et par anime
user_rating_counts = user_score.groupby('user_id').size()
anime_rating_counts = user_score.groupby('anime_id').size()

# Filtrer les utilisateurs avec au moins 10 notes et les animes avec au moins 100,000 notes
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
filtered_animes = anime_rating_counts[anime_rating_counts >= min_anime_ratings].index

# Restreindre le dataset aux utilisateurs et animes filtrés
filtered_scores = user_score[user_score['user_id'].isin(filtered_users) & user_score['anime_id'].isin(filtered_animes)]

In [5]:
user_rating_counts

user_id
1          262
4          284
9           65
20         105
23         282
          ... 
1291057    240
1291079     94
1291085     59
1291087    210
1291097      3
Length: 270033, dtype: int64

In [6]:
filtered_users

Index([      1,       4,       9,      20,      23,      37,      47,      48,
            53,      66,
       ...
       1291015, 1291017, 1291021, 1291029, 1291033, 1291049, 1291057, 1291079,
       1291085, 1291087],
      dtype='int64', name='user_id', length=117568)

In [7]:
filtered_scores

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
6,1,Xinil,53,Ai Yori Aoshi,7
7,1,Xinil,47,Akira,5
8,1,Xinil,591,Amaenaide yo!!,6
...,...,...,...,...,...
24325178,1291087,Oblongata,14813,Yahari Ore no Seishun Love Comedy wa Machigatt...,8
24325179,1291087,Oblongata,8861,"Yosuga no Sora: In Solitude, Where We Are Leas...",3
24325183,1291087,Oblongata,10578,C³,6
24325184,1291087,Oblongata,12549,"Dakara Boku wa, H ga Dekinai.",4


In [8]:
import gc

# Supprimer la variable
del user_score

# Forcer le garbage collector à libérer la mémoire
gc.collect()

# filtered_scores devient user_score on prend juste un échantillon des 1000 premiers animés
filtered_scores = filtered_scores.head(1000)

In [9]:
# Vérifier la taille du dataset après filtrage
print(f"Nombre d'utilisateurs filtrés : {len(filtered_users)}")
print(f"Nombre d'animes filtrés : {len(filtered_animes)}")
print(f"Nombre de notes après filtrage : {len(filtered_scores)}")

Nombre d'utilisateurs filtrés : 117568
Nombre d'animes filtrés : 598
Nombre de notes après filtrage : 1000


### Collaborative Filtering

In [10]:
# Créer le dataset pour Surprise
reader = Reader(rating_scale=(1, 10))  # Assuming rating scale is from 1 to 10
data = Dataset.load_from_df(filtered_scores[['user_id', 'anime_id', 'rating']], reader)

In [11]:
data

In [12]:
train, test = train_test_split(data, test_size=0.2)

In [13]:
print(type(train))
print(type(test))

<class 'surprise.trainset.Trainset'>
<class 'list'>


In [14]:
cpt=0
for r in train.all_ratings():
  print(r)
  cpt+=1
  if cpt > 20:
    break

(0, 0, 8.0)
(0, 1, 7.0)
(0, 10, 6.0)
(0, 24, 6.0)
(0, 31, 5.0)
(0, 39, 7.0)
(0, 41, 8.0)
(0, 48, 8.0)
(0, 50, 9.0)
(0, 53, 7.0)
(0, 59, 8.0)
(0, 60, 8.0)
(0, 63, 2.0)
(0, 5, 7.0)
(0, 64, 7.0)
(0, 69, 6.0)
(0, 29, 7.0)
(0, 75, 9.0)
(0, 79, 7.0)
(0, 23, 7.0)
(0, 94, 8.0)


### Construction du jeu antitest

In [15]:
full_train = data.build_full_trainset()
antitest = full_train.build_anti_testset()

In [16]:
print(antitest)

[(1, 298, 7.541), (1, 5525, 7.541), (1, 6682, 7.541), (1, 11759, 7.541), (1, 101, 7.541), (1, 102, 7.541), (1, 22199, 7.541), (1, 4744, 7.541), (1, 25013, 7.541), (1, 22729, 7.541), (1, 27655, 7.541), (1, 6547, 7.541), (1, 9989, 7.541), (1, 11433, 7.541), (1, 11111, 7.541), (1, 24833, 7.541), (1, 21995, 7.541), (1, 9919, 7.541), (1, 12967, 7.541), (1, 477, 7.541), (1, 9736, 7.541), (1, 5342, 7.541), (1, 2251, 7.541), (1, 3901, 7.541), (1, 6347, 7.541), (1, 7674, 7.541), (1, 10030, 7.541), (1, 2986, 7.541), (1, 22789, 7.541), (1, 10396, 7.541), (1, 13535, 7.541), (1, 20787, 7.541), (1, 7059, 7.541), (1, 10490, 7.541), (1, 31043, 7.541), (1, 31964, 7.541), (1, 33486, 7.541), (1, 36456, 7.541), (1, 10719, 7.541), (1, 14967, 7.541), (1, 6772, 7.541), (1, 8514, 7.541), (1, 14345, 7.541), (1, 31478, 7.541), (1, 1536, 7.541), (1, 10163, 7.541), (1, 12293, 7.541), (1, 5356, 7.541), (1, 10578, 7.541), (1, 28999, 7.541), (1, 10800, 7.541), (1, 14397, 7.541), (1, 4186, 7.541), (1, 14741, 7.541), 

### Implémentation des métriques

### Top N

In [17]:
def top_n(predictions, N, rating_min):
  df = pd.DataFrame(predictions)
  df = df[df["est"] >= rating_min].drop(["r_ui", "details"], axis=1)
  df = df.sort_values(by="est", ascending=False)
  df = df.groupby("uid").head(N)
  df = df.groupby("uid").agg({"iid": list, "est": list})
  df.columns = ["topn", "est"]
  return df

In [18]:
# topn = top_n(predictions, N=100, rating_min=0)

### Cumulative Hit Rate
### Le hit rate correspond au cumulative hit rate avec un rating_min=0.

In [19]:
def count_hit(row):
  if isinstance(row["topn"], list):
    return 1 if row["iid"] in row["topn"] else 0
  else:
    return 0

def cumulative_hit_rate(topn, test):
  df_test = pd.DataFrame(test, columns = ["uid", "iid", "r_ui"])
  df_test =  df_test.set_index("uid")
  df_test["topn"] = topn["topn"]


  # Calcul du nombre de hits:
  hits = df_test.apply(count_hit, axis=1).sum()
  res = hits/len(test)

  return res

In [20]:
# cumulative_hit_rate(topn, test)

### Average Reciprocal Hit Rate

In [21]:
def get_weight(row):
  """
  Si iid est dans le top N, on renvoie 1/p où p est la position de iid dans le top N
  Sinon, on renvoie 0
  """
  if isinstance(row["topn"], list):
    if row["iid"] in row["topn"]:
      return 1/(row["topn"].index(row["iid"]) + 1)
    else:
      return 0
  else:
    return 0

def average_reciprocal_hit_rate(topn, test):
  df_test = pd.DataFrame(test, columns = ["uid", "iid", "r_ui"])
  df_test =  df_test.set_index("uid")
  df_test["topn"] = topn["topn"]
  weights = df_test.apply(get_weight, axis=1).sum()
  res = weights/len(test)

  return res

In [22]:
# average_reciprocal_hit_rate(topn, test)

### User Coverage

In [23]:
def check_threshold(row, threshold):
  """
  Retourne 1 s'il existe une note estimée au dessus du seuil, 0 sinon
  """
  if isinstance(row["est"], list):
    return 1 if row["est"][0] >= threshold else 0

  else:
    return 0

def user_coverage(topn, threshold=0):
  coverage = topn.apply(check_threshold, threshold=threshold, axis=1).sum()
  res = coverage/len(test)

  return res

In [24]:
# user_coverage(topn, threshold=5)

### Création d'une fonction d'évaluation des modèles

In [25]:
import json
performances = {}

def evaluate_model(model, name, test, antitest=None, N=10, rating_min=0, threshold=0):
  predictions = model.test(test)
  mae = accuracy.mae(predictions, verbose=False)
  rmse = accuracy.rmse(predictions, verbose=False)
  performances[name] = {"MAE": mae, "RMSE": rmse}

  if antitest is not None:
    predictions = model.test(antitest)
    topn = top_n(N=N, predictions=predictions, rating_min=rating_min)
    chr = cumulative_hit_rate(topn=topn, test=test)
    arhr = average_reciprocal_hit_rate(topn=topn, test=test)
    uc = user_coverage(topn, threshold=threshold)

    performances[name]["Cumulative Hit Rate"] = chr
    performances[name]["Average Reciprocal Hit Rate"] = arhr
    performances[name]["User Coverage"] = uc

  print(json.dumps(performances, indent=2))

In [26]:
# evaluate_model(model=random_model, name="Modele gaussien", test=test, antitest=antitest, threshold=4, N=50, rating_min=2.5)

### Collaborative Filtering

### KNN basique (KNNBasic)

In [27]:
hyperparameters_knn_basic = {
    "k" : [20, 30, 40],
    "min_k" : [1, 3, 5],
    "sim_options": {
        "name": ["cosine", "msd", "pearson"],
        "user_based": [True]
    }
}

In [28]:
gs_knn_basic = GridSearchCV(
  KNNBasic,
  param_grid=hyperparameters_knn_basic,
  cv = 2,
  measures=['mae'],
)

In [29]:
gs_knn_basic.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine

In [30]:
best_knn_basic = gs_knn_basic.best_estimator["mae"]
print(best_knn_basic)

In [31]:
best_knn_basic.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [32]:
best_knn_basic.test(test)

[Prediction(uid=23, iid=13161, r_ui=7.0, est=7.55375, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=48, iid=97, r_ui=9.0, est=8.00547761883207, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=4, iid=123, r_ui=6.0, est=7.55375, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=1, iid=12355, r_ui=8.0, est=7.55375, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=47, iid=4654, r_ui=8.0, est=7.55375, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=23, iid=2904, r_ui=8.0, est=7.55375, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=20, iid=263, r_ui=9.0, est=7.55375, details={'was_impossible': True, 'reason': 'Not enough neighbors.'}),
 Prediction(uid=48, iid=1535, r_ui=7.0, est=8.002006622039215, details={'actual_k': 3, 'was_impossible': False}),
 Prediction

In [33]:
# On observe beaucoup de "was impossible = True" le modèle n'arrive pas à faire des prédictions
trainset = best_knn_basic.trainset
antitest = trainset.build_anti_testset()

In [34]:
evaluate_model(model=best_knn_basic, name="KNN Basic", test=test, antitest=antitest,threshold=4, N=50, rating_min=5)

{
  "KNN Basic": {
    "MAE": 1.2590639819050304,
    "RMSE": 1.5634364772503673,
    "Cumulative Hit Rate": 0.24,
    "Average Reciprocal Hit Rate": 0.018800024264368897,
    "User Coverage": 0.05
  }
}


### KNN avec moyennes (KNNWithMeans)

In [35]:
hyperparameters_knn_means = {
    "k" : [20, 30, 40],
    "min_k" : [1, 3, 5],
    "sim_options": {
        "name": ["cosine", "msd", "pearson"],
        "user_based": [True]
    }
}

In [36]:
gs_knn_means = GridSearchCV(
  KNNWithMeans,
  param_grid=hyperparameters_knn_means,
  cv = 2,
  measures=['mae'],
)

In [37]:
gs_knn_means.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine

In [38]:
best_knn_means = gs_knn_means.best_estimator["mae"]

In [39]:
best_knn_means.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [40]:
evaluate_model(model=best_knn_means, name="KNN avec moyennes", test=test, antitest=antitest,threshold=4, N=50, rating_min=5)

{
  "KNN Basic": {
    "MAE": 1.2590639819050304,
    "RMSE": 1.5634364772503673,
    "Cumulative Hit Rate": 0.24,
    "Average Reciprocal Hit Rate": 0.018800024264368897,
    "User Coverage": 0.05
  },
  "KNN avec moyennes": {
    "MAE": 1.1631594102502676,
    "RMSE": 1.485557703361385,
    "Cumulative Hit Rate": 0.225,
    "Average Reciprocal Hit Rate": 0.021470230784661065,
    "User Coverage": 0.05
  }
}


### KNN centré réduit (KNNWithZScore)

In [41]:
hyperparameters_knn_zscore = {
    "k" : [20, 30, 40],
    "min_k" : [1, 3, 5],
    "sim_options": {
        "name": ["cosine", "msd", "pearson"],
        "user_based": [True]
    }
}

In [42]:
gs_knn_zscore = GridSearchCV(
  KNNWithZScore,
  param_grid=hyperparameters_knn_zscore,
  cv = 2,
  measures=['mae'],
)

In [43]:
gs_knn_zscore.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine

In [44]:
best_knn_zscore = gs_knn_zscore.best_estimator["mae"]

In [45]:
best_knn_zscore.fit(train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [46]:
evaluate_model(model=best_knn_zscore, name="KNN centré réduit", test=test, antitest=antitest, threshold=4, N=50, rating_min=5)

{
  "KNN Basic": {
    "MAE": 1.2590639819050304,
    "RMSE": 1.5634364772503673,
    "Cumulative Hit Rate": 0.24,
    "Average Reciprocal Hit Rate": 0.018800024264368897,
    "User Coverage": 0.05
  },
  "KNN avec moyennes": {
    "MAE": 1.1631594102502676,
    "RMSE": 1.485557703361385,
    "Cumulative Hit Rate": 0.225,
    "Average Reciprocal Hit Rate": 0.021470230784661065,
    "User Coverage": 0.05
  },
  "KNN centr\u00e9 r\u00e9duit": {
    "MAE": 1.1887512321191276,
    "RMSE": 1.5267650726402253,
    "Cumulative Hit Rate": 0.245,
    "Average Reciprocal Hit Rate": 0.02455819388364281,
    "User Coverage": 0.05
  }
}


### KNN en tenant compte des biais $b_u$ et $b_i$ (KNNBaseline)

In [47]:
hyperparameters_knn_bu_bi = {
    "k" : [20, 30, 40],
    "min_k" : [1, 3, 5],
    "sim_options": {
        "name": ["cosine", "msd", "pearson"],
        "user_based": [True]
    },
    "bsl_options": {
        "method": ["sgd"],
        "learning_rate": [0.005, 0.01]
    }
}

In [48]:
gs_knn_bu_bi = GridSearchCV(
  KNNBaseline,
  param_grid=hyperparameters_knn_bu_bi,
  cv = 2,
  measures=['mae'],
)

In [49]:
gs_knn_bu_bi.fit(data)

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pea

In [50]:
best_knn_bu_bi = gs_knn_bu_bi.best_estimator["mae"]

In [51]:
best_knn_bu_bi.fit(train)

Estimating biases using sgd...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [52]:
evaluate_model(model=best_knn_bu_bi, name="KNN avec les biais bu et bi", test=test, antitest=antitest, threshold=4, N=50, rating_min=5)

{
  "KNN Basic": {
    "MAE": 1.2590639819050304,
    "RMSE": 1.5634364772503673,
    "Cumulative Hit Rate": 0.24,
    "Average Reciprocal Hit Rate": 0.018800024264368897,
    "User Coverage": 0.05
  },
  "KNN avec moyennes": {
    "MAE": 1.1631594102502676,
    "RMSE": 1.485557703361385,
    "Cumulative Hit Rate": 0.225,
    "Average Reciprocal Hit Rate": 0.021470230784661065,
    "User Coverage": 0.05
  },
  "KNN centr\u00e9 r\u00e9duit": {
    "MAE": 1.1887512321191276,
    "RMSE": 1.5267650726402253,
    "Cumulative Hit Rate": 0.245,
    "Average Reciprocal Hit Rate": 0.02455819388364281,
    "User Coverage": 0.05
  },
  "KNN avec les biais bu et bi": {
    "MAE": 1.082168948536124,
    "RMSE": 1.4308496700781395,
    "Cumulative Hit Rate": 0.225,
    "Average Reciprocal Hit Rate": 0.032892209021112395,
    "User Coverage": 0.05
  }
}


##### 1. Factorisation de Matrice sans Feedbacks Implicites : SVD (Singular Value Decomposition)
##### La méthode SVD est l'une des plus populaires pour la factorisation de matrice. Elle ne tient pas compte des feedbacks implicites et se base uniquement sur les ratings explicites des utilisateurs.

##### Étape 1 : Définir un dictionnaire d’hyperparamètres pour SVD
##### Les hyperparamètres principaux pour SVD incluent :

##### n_factors : Le nombre de facteurs latents.
##### n_epochs : Le nombre d'itérations pour entraîner le modèle.
##### lr_all : Le taux d'apprentissage pour la descente de gradient.
##### reg_all : Le coefficient de régularisation.

In [53]:
param_grid = {
    'n_factors': [50, 100, 150],   # Nombre de facteurs latents
    'n_epochs': [20, 30, 40],      # Nombre d'itérations
    'lr_all': [0.002, 0.005],      # Taux d'apprentissage
    'reg_all': [0.02, 0.1]         # Régularisation
}

In [54]:
# Initialiser GridSearchCV pour le modèle SVD
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)

# Appliquer la validation croisée sur le dataset
gs.fit(data)

# Afficher les meilleurs hyperparamètres
print(gs.best_params['rmse'])

{'n_factors': 100, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.02}


In [55]:
# Récupérer les meilleurs hyperparamètres
best_params = gs.best_params['rmse']

# Entraîner le modèle avec ces hyperparamètres
SVDModel = SVD(n_factors=best_params['n_factors'],
           n_epochs=best_params['n_epochs'],
           lr_all=best_params['lr_all'],
           reg_all=best_params['reg_all'])

# Entraîner le modèle sur le jeu d'entraînement
SVDModel.fit(train)

# Tester le modèle sur le jeu de test
predictions = SVDModel.test(test)

# évaluer le modèle (Calculer et afficher les métriques)
# Calculer les métriques
rmse = accuracy.rmse(predictions, verbose=True)
mae = accuracy.mae(predictions, verbose=True)

RMSE: 1.4727
MAE:  1.1054


##### 2. Factorisation de Matrice avec Feedbacks Implicites : SVD++
##### Le modèle SVD++ prend en compte à la fois les feedbacks explicites et implicites. Cela inclut des interactions comme des clics, des ajouts à une liste de souhaits, des vues de films, etc., en plus des évaluations explicites données par les utilisateurs.
##### Étape 1 : Définir un dictionnaire d’hyperparamètres pour SVD++
##### Les hyperparamètres de SVD++ sont similaires à ceux de SVD, avec des facteurs supplémentaires pour les feedbacks implicites :
##### Étape 2 : Validation croisée avec GridSearchCV pour SVD++
##### La validation croisée se fait de la même manière que pour SVD, mais avec le modèle SVD++ :
##### Étape 3 : Mesurer l'efficacité sur le jeu de test (MAE et RMSE)
##### Vous pouvez maintenant entraîner le modèle SVD++ avec les meilleurs hyperparamètres et mesurer son efficacité :

In [56]:
# Initialiser GridSearchCV pour le modèle SVD++
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

# Appliquer la validation croisée
gs.fit(data)

# Afficher les meilleurs hyperparamètres
print(gs.best_params['rmse'])

{'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.1}


In [57]:
# Récupérer les meilleurs hyperparamètres
best_params = gs.best_params['rmse']

# Entraîner le modèle avec ces hyperparamètres
SVDPPModel = SVDpp(n_factors=best_params['n_factors'],
             n_epochs=best_params['n_epochs'],
             lr_all=best_params['lr_all'],
             reg_all=best_params['reg_all'])

# Entraîner et tester le modèle
SVDPPModel.fit(train)
predictions = SVDPPModel.test(test)

# Calculer les métriques
rmse = accuracy.rmse(predictions, verbose=True)
mae = accuracy.mae(predictions, verbose=True)

RMSE: 1.4436
MAE:  1.0919


### One Slope

In [58]:
# Créer et entraîner le modèle Slope One
SlopeOneModel = SlopeOne()
SlopeOneModel.fit(train)

In [59]:
# Faire des prédictions sur le jeu de test
predictions = SlopeOneModel.test(test)

# Calculer le MAE et le RMSE pour évaluer les performances du modèle
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

MAE:  1.2049
RMSE: 1.5952


### Co Clustering

In [60]:
# Définir un dictionnaire d'hyperparamètres à explorer
hyperparameters = {
    'n_cltr_u': [3, 5, 7],  # Nombre de clusters d'utilisateurs
    'n_cltr_i': [3, 5, 7],  # Nombre de clusters d'items
    'n_epochs': [10, 20, 30]  # Nombre d'itérations pour l'entraînement
}

# Appliquer GridSearchCV pour trouver les meilleurs hyperparamètres
gs = GridSearchCV(CoClustering, hyperparameters, measures=['rmse', 'mae'], cv=2)
gs.fit(data)

# Afficher les meilleurs hyperparamètres et les résultats associés
print("Meilleurs hyperparamètres:", gs.best_params['rmse'])
print("Meilleur score RMSE:", gs.best_score['rmse'])

Meilleurs hyperparamètres: {'n_cltr_u': 3, 'n_cltr_i': 7, 'n_epochs': 20}
Meilleur score RMSE: 1.5988770383292428


In [61]:
# Entraîner le modèle avec les meilleurs hyperparamètres
best_algo = gs.best_estimator['rmse']
best_algo.fit(train)

In [62]:
# Faire des prédictions sur le jeu de test
predictions = best_algo.test(test)

# Mesurer le MAE et le RMSE
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

MAE:  1.2773
RMSE: 1.6214


### Implémenter une fonction prenant un identifiant utilisateur en entrée et qui retournera le topN pour cet utilisateur sous la forme "titre" - "note moyenne"  - "nombre de notes".

In [63]:
# 1. Calculer le nombre de notes par anime
nb_ratings_per_anime = filtered_scores.groupby(by="anime_id").size().rename("nb_ratings")

# 2. Joindre le nombre de notes au dataset principal
# En utilisant merge pour ajouter la colonne nb_ratings au dataset score_dataset
score_dataset_with_ratings = filtered_scores.merge(nb_ratings_per_anime, on="anime_id", how="left")

# Optionnel : Vérifier le résultat
print(score_dataset_with_ratings.head())

   user_id Username  anime_id     Anime Title  rating  nb_ratings
0        1    Xinil        21       One Piece       9           3
1        1    Xinil        48     .hack//Sign       7           4
2        1    Xinil        53   Ai Yori Aoshi       7           4
3        1    Xinil        47           Akira       5           2
4        1    Xinil       591  Amaenaide yo!!       6           2


In [64]:
# 1. Calculer la note moyenne par anime
average_ratings_per_anime = score_dataset_with_ratings.groupby(by="anime_id")["rating"].mean().rename("average_ratings")

# 2. Joindre la note moyenne au dataset principal
# En utilisant merge pour ajouter la colonne average_ratings au dataset score_dataset
score_dataset_with_avg_ratings = score_dataset_with_ratings.merge(average_ratings_per_anime, on="anime_id", how="left")

# Optionnel : Vérifier le résultat
print(score_dataset_with_avg_ratings.head())

   user_id Username  anime_id     Anime Title  rating  nb_ratings  \
0        1    Xinil        21       One Piece       9           3   
1        1    Xinil        48     .hack//Sign       7           4   
2        1    Xinil        53   Ai Yori Aoshi       7           4   
3        1    Xinil        47           Akira       5           2   
4        1    Xinil       591  Amaenaide yo!!       6           2   

   average_ratings  
0         8.333333  
1         7.000000  
2         7.250000  
3         7.500000  
4         5.500000  


In [65]:
dict_title_to_id = dict(zip(score_dataset_with_avg_ratings["Anime Title"], score_dataset_with_avg_ratings["anime_id"] ))
dict_id_to_title = {v : k for k, v in dict_title_to_id.items()}

In [66]:
# Créer le DataFrame df_0 avec les caractéristiques de chaque anime
df_0 = pd.DataFrame({
    'anime_id': nb_ratings_per_anime.index,  # Identifiant de l'anime
    'anime_title': nb_ratings_per_anime.index.map(dict_id_to_title),  # Titre de l'anime
    'average_ratings': average_ratings_per_anime,  # Note moyenne
    'nb_ratings': nb_ratings_per_anime  # Nombre de notes
})

# Supprimer les animes avec des valeurs manquantes (si nécessaire)
df_0 = df_0.dropna()

# Afficher les premières lignes de df_0 pour vérifier
print(df_0.head())

          anime_id                      anime_title  average_ratings  \
anime_id                                                               
1                1                     Cowboy Bebop         8.833333   
5                5  Cowboy Bebop: Tengoku no Tobira         7.833333   
6                6                           Trigun         8.500000   
7                7               Witch Hunter Robin         8.000000   
15              15                     Eyeshield 21         7.000000   

          nb_ratings  
anime_id              
1                  6  
5                  6  
6                  4  
7                  3  
15                 2  


In [67]:
from sklearn.neighbors import NearestNeighbors

def recommander_animes_utilisateur(user_id, score_dataset, df_0, model, n_recommendations=10):
    # Filtrer le dataset pour obtenir les animes notés par l'utilisateur
    animes_utilisateur = score_dataset[score_dataset['user_id'] == user_id]
    
    # Si l'utilisateur n'a pas d'animes, retourner un message vide
    if animes_utilisateur.empty:
        print(f"Aucun anime trouvé pour l'utilisateur {user_id}")
        return []
    
    # Choisir un anime de l'utilisateur pour la recommandation (ici, le premier)
    anime_id = animes_utilisateur.iloc[0]['anime_id']
    
    # Extraire le vecteur des caractéristiques de l'anime choisi par l'utilisateur
    film_1_vector = df_0[df_0['anime_id'] == anime_id][['average_ratings', 'nb_ratings']]

    # Utiliser le modèle KNN pour trouver les voisins les plus proches (recommandations)
    distance, neig = model.kneighbors(film_1_vector.to_numpy(), n_neighbors=n_recommendations + 1)  # +1 pour ignorer l'anime lui-même
    
    print("Anime Name : ")
    print(df_0[df_0['anime_id'] == anime_id])

    # Créer une liste pour stocker les recommandations
    recommendations = []

    # Parcourir les voisins et extraire les titres d'animes recommandés
    for i in range(1, len(distance[0])):  # Commencer à 1 pour ignorer l'anime de départ
        titre = df_0.iloc[neig[0][i]]['anime_title']
        avg_rating = df_0.iloc[neig[0][i]]['average_ratings']
        nb_ratings = df_0.iloc[neig[0][i]]['nb_ratings']
        
        # Ajouter la recommandation à la liste
        recommendations.append((titre, avg_rating, nb_ratings))
        print(f"{titre} recommandé avec une note moyenne de {avg_rating:.2f} et {nb_ratings} notes.")

    return recommendations

In [68]:
# Sélectionner les colonnes de caractéristiques (note moyenne et nombre de notes)
features = df_0[['average_ratings', 'nb_ratings']]

# Créer et entraîner le modèle KNN
model_knn = NearestNeighbors(n_neighbors=11, algorithm='auto')
model_knn.fit(features)

# Exemple d'appel de la fonction de recommandation
topN_recommendations = recommander_animes_utilisateur(user_id=1, score_dataset=score_dataset_with_avg_ratings, df_0=df_0, model=model_knn, n_recommendations=10)

Anime Name : 
          anime_id anime_title  average_ratings  nb_ratings
anime_id                                                   
21              21   One Piece         8.333333           3
Koukaku Kidoutai: Stand Alone Complex recommandé avec une note moyenne de 8.33 et 3 notes.
Sword Art Online recommandé avec une note moyenne de 8.33 et 3 notes.
Hajime no Ippo: New Challenger recommandé avec une note moyenne de 8.33 et 3 notes.
Mushishi recommandé avec une note moyenne de 8.33 et 3 notes.
One Piece recommandé avec une note moyenne de 8.33 et 3 notes.
One Punch Man recommandé avec une note moyenne de 8.33 et 3 notes.
Tengen Toppa Gurren Lagann recommandé avec une note moyenne de 8.67 et 3 notes.
Higurashi no Naku Koro ni recommandé avec une note moyenne de 8.67 et 3 notes.
Shingetsutan Tsukihime recommandé avec une note moyenne de 8.67 et 3 notes.
Fruits Basket recommandé avec une note moyenne de 8.67 et 3 notes.


C:\Users\anasa\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
